# Data retrieval with Obspy

Here we will use the python pacakge *ObsPy* to access the [FDSN](2_Web-Based.ipynb#FDSN) services.

Obspy has several available clients - one of them is the FDSN, which we will use here. The [documentation](https://docs.obspy.org/packages/obspy.clients.fdsn.html) for more details.

### Tabel of content:
- [Waveform](#Waveform)
- [Event](#Event)
- [Station](#Station)
- [Routing](#Routing)
- [Summary](#Summary)

In [ ]:
from obspy import UTCDateTime
from obspy.clients.fdsn import Client, header

We can check which agencies are available:

In [ ]:
for key in sorted(header.URL_MAPPINGS.keys()):
    print("{0:<11} {1}".format(key,  header.URL_MAPPINGS[key]))

Here we select *IRIS*. 

In [ ]:
client = Client("IRIS")

## Waveform
<a id='Waveform'></a>
We use the [get_waveforms()](https://docs.obspy.org/packages/autogen/obspy.clients.fdsn.client.Client.get_waveforms.html#obspy.clients.fdsn.client.Client.get_waveforms) function, which needs (atleast) station information (Network, Station, Location and Channel - NSLC) and a time span. 

The results are stored in a *stream* object. This contains the waveform seperated by NSLC.

In [ ]:
t = UTCDateTime("2011-03-11T05:46:23")  # Tohoku
st = client.get_waveforms("II", "PFO", "*", "LHZ",
                          t + 10 * 60, t + 30 * 60)
print(st)
st.plot()
print('Finished')

In [ ]:
print(st[0])
print(st[0].id)
print(st[0].data)
print(st[0].stats)

There is the option to directly download the response of sensors as well and remove them in an easy fashion. 

In [ ]:
t = UTCDateTime("2011-03-11T05:46:23")  # Tohoku
st = client.get_waveforms("II", "PFO", "*", "LHZ",
                          t + 10 * 60, t + 30 * 60, attach_response=True)
st.remove_response()
st.plot()
print('Finished')

## Event and Catalogs
<a id='Event'></a>
To request  event and catalog information, with [get_events()](https://docs.obspy.org/packages/autogen/obspy.clients.fdsn.client.Client.get_events.html#obspy.clients.fdsn.client.Client.get_events), you should atleast define a time span, otherwise you download *the whole internet* or all events. Additionaly, you can define from which catalog/site/organization you want to request the event information. As a result you get a Obspy catalog object, including all requested events.

In [ ]:
starttime = UTCDateTime("2014-07-06")
endtime = UTCDateTime("2014-07-07")

cat = client.get_events(starttime=starttime, endtime=endtime,
                        minmagnitude=5, catalog="ISC")

print(cat)  

cat.plot()
print('Finished')

In [ ]:
print(cat[0])
print(cat[0].event_type)
print(cat[0].magnitudes)

## Station meta data
<a id='Station'></a>
Similiar to the event request we can also request station meta data using [get_stations()](https://docs.obspy.org/packages/autogen/obspy.clients.fdsn.client.Client.get_stations.html#obspy.clients.fdsn.client.Client.get_stations). The result is an inventory object, containing all stations.

We need to keep the *level* option in mind, we may have to much unnecessary informaton or we may restrict our data too much.

In [ ]:
starttime = UTCDateTime("2014-07-06")
endtime = UTCDateTime("2014-07-07")

inventory = client.get_stations(
    network="IU",
    station="A*",
    level='station',  # or 'network', 'response'
    starttime=starttime,
    endtime=endtime)

print(inventory)  

inventory.plot()
print('Finished')

### Store information to file
All of the three mentioned functions can directly store its output into a file.

In [ ]:
client = Client("IRIS")

starttime = UTCDateTime("2019-07-06 03:00:00")
endtime = starttime + 6000
net = "II"
sta = "PFO"


client.get_events(starttime=starttime, endtime=endtime, minmagnitude=5,
                  filename="/tmp/events.xml")
client.get_stations(network=net, station=sta, level="station",
                    filename="/tmp/stations.xml")
client.get_waveforms(network=net, station=sta, location='*', channel='LHZ',
                     starttime=starttime, endtime=endtime,
                     filename="/tmp/waveforms.mseed")

In [ ]:
!ls -lrt /tmp/

## Routing
<a id='Routing'></a>
The descripted way has the disadvantage that one has to approach each of the centers manually. Especially, if you want to have a complete set of stations or if you don't know which center is responsible for which station. The [routing](https://docs.obspy.org/packages/obspy.clients.fdsn.html#basic-routing-clients-usage) makes those things easier as services are combined and they are automatically searched for your requests.

In Obspy, there are two Routing options:
- IRIS Federator
- EIDAWS Routing Service

In [ ]:
from obspy.clients.fdsn import RoutingClient
client = RoutingClient("iris-federator")

st = client.get_waveforms(
    channel="LHZ", starttime=UTCDateTime(2017, 1, 1),
    endtime=UTCDateTime(2017, 1, 1, 0, 5), latitude=10,
    longitude=10, maxradius=25)  

print(st)  

# Summary
In this notebook we have learned the usage of Obspy with FDSN to request and download waveform, station and event data.